Installing dependencies. You might need to tweak the CMAKE_ARGS for the `llama-cpp-python` pip package.

In [1]:
# GPU llama-cpp-python; Starting from version llama-cpp-python==0.1.79, it supports GGUF
!CMAKE_ARGS="-DLLAMA_CUBLAS=on " pip install 'llama-cpp-python>=0.1.79' --force-reinstall --upgrade --no-cache-dir
# For download the models
!pip install huggingface_hub
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 2.4 MB/s eta 0:00:000:00:01m eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 3.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [37 lines of output]
      *** scikit-build-core 0.7.0 using CMake 3.22.1 (wheel)
      *** Configuring CMake...
      loading initial cache file /tmp/tmpmgsfdp15/build/CMakeInit.txt
      -- The C compiler identification is GNU 11.4.0
      -- The CXX compiler identification is GNU 11.4.0
      -- Detecting C compiler ABI info
      -- Detecting C compiler ABI 

We start by downloading an instruction-finetuned Mistral model, which we will ask to classify model outputs for us.

In [2]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# This config has been tested on an RTX 3080 (VRAM of 16GB).
# you might need to tweak with respect to your hardware.
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=16, # CPU cores
    n_batch=8000, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192, # Context window
    logits_all=True
)

run_on_test = False # whether this baseline system is ran on the test splits or the val splits

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3080 Laptop GPU, compute capability 8.6
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /home/mickus/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q6_K     [  4096,  40

Running on the model-aware track data

In [3]:
# change paths appropriately
# make sure the output filename is the same as the reference filename for the scoring program
path_val_model_aware = "path/to/reference/val.model-aware.json"
path_val_model_aware_output = "path/to/output/val.model-aware.json" 

from datasets import load_dataset
import json
import random
import numpy as np
import tqdm.notebook as tqdm
seed_val = 442
random.seed(seed_val)
np.random.seed(seed_val)

# simple JSON loading
with open(path_val_model_aware, 'r') as istr:
    data_val_all = json.load(istr)
num_sample = len(data_val_all)
print(num_sample)

output_json = []
labels = ["Not Hallucination", "Hallucination"]
"""
SelfCheckGPT Usage: (LLM) Prompt
https://github.com/potsawee/selfcheckgpt
Context: {}
Sentence: {}
Is the sentence supported by the context above?
Answer Yes or No:
"""
for i in tqdm.trange(num_sample):
    task = str(data_val_all[i]['task'])
    if run_on_test:
        # test splits will contain ids to ensure correct alignment before scoring
        id = int(data_val_all[i]['id'])
    hyp = str(data_val_all[i]['hyp'])
    src = str(data_val_all[i]['src'])
    tgt = str(data_val_all[i]['tgt'])

    if task == "PG":
        context = f"Context: {src}"
    else: #i.e. task == "MT" or task == "DM":
        context = f"Context: {tgt}"

    sentence = f"Sentence: {hyp}"
    message = f"{context}\n{sentence}\nIs the Sentence supported by the Context above? Answer using ONLY yes or no:"
    prompt = f"<s>[INST] {message} [/INST]"

    response = lcpp_llm(
        prompt=prompt,
        temperature= 0.0,
        logprobs=1,
    )
    answer = str(response["choices"][0]["text"]).strip().lower()
    if answer.startswith("yes"):
        output_label = "Not Hallucination"
        prob = 1-float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if answer.startswith("no"):
        output_label = "Hallucination"
        prob = float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if not answer.startswith("no") and not answer.startswith("yes"):
        idx_random = random.randint(0,len(labels)-1)
        output_label = labels[idx_random]
        prob = float(0.5)

    item_to_json = {"label":output_label, "p(Hallucination)":prob}
    if run_on_test:
        item_to_json['id'] = id
    
    output_json.append(item_to_json)


f = open(path_val_model_aware_output, 'w', encoding='utf-8')
json.dump(output_json, f)
f.close()
print("done")

501


  0%|          | 0/501 [00:00<?, ?it/s]


llama_print_timings:        load time =     205.56 ms
llama_print_timings:      sample time =       7.58 ms /    35 runs   (    0.22 ms per token,  4620.46 tokens per second)
llama_print_timings: prompt eval time =     205.47 ms /    51 tokens (    4.03 ms per token,   248.21 tokens per second)
llama_print_timings:        eval time =    1714.84 ms /    34 runs   (   50.44 ms per token,    19.83 tokens per second)
llama_print_timings:       total time =    2055.84 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =     205.56 ms
llama_print_timings:      sample time =       7.25 ms /    36 runs   (    0.20 ms per token,  4964.83 tokens per second)
llama_print_timings: prompt eval time =     198.39 ms /    42 tokens (    4.72 ms per token,   211.71 tokens per second)
llama_print_timings:        eval time =    1363.09 ms /    35 runs   (   38.95 ms per token,    25.68 tokens per second)
llama_print_timings:       total time =    1681.81 ms
Llama.generate: prefix-

done


Running on the model-agnostic track data

In [4]:
# change paths appropriately
# make sure the output filename is the same as the reference filename for the scoring program
path_val_model_agnostic = "path/to/reference/val.model-agnostic.json"
path_val_model_agnostic_output = "path/to/output/val.model-agnostic.json"

from datasets import load_dataset
import json
import random
import numpy as np
import tqdm.notebook as tqdm
seed_val = 442
random.seed(seed_val)
np.random.seed(seed_val)

# alternatively, one can use HuggingFace's library to load the data
dataset = load_dataset('json', data_files={'val':path_val_model_agnostic})
data_val_all = dataset['val']
num_sample = len(data_val_all)
print(dataset)
print(num_sample)

output_json = []
labels = ["Not Hallucination", "Hallucination"]
"""
SelfCheckGPT Usage: (LLM) Prompt
https://github.com/potsawee/selfcheckgpt
Context: {}
Sentence: {}
Is the sentence supported by the context above?
Answer Yes or No:
"""
for i in tqdm.trange(num_sample):
    # label = str(data_val_all[i]['label'])
    task = str(data_val_all[i]['task'])
    if run_on_test:
        # test splits will contain ids to ensure correct alignment before scoring
        id = int(data_val_all[i]['id'])
    hyp = str(data_val_all[i]['hyp'])
    src = str(data_val_all[i]['src'])
    tgt = str(data_val_all[i]['tgt'])

    if task == "PG":
        context = f"Context: {src}"
    else: #i.e. task == "MT" or task == "DM":
        context = f"Context: {tgt}"

    sentence = f"Sentence: {hyp}"
    message = f"{context}\n{sentence}\nIs the Sentence supported by the Context above? Answer using ONLY yes or no:"
    prompt = f"<s>[INST] {message} [/INST]"

    response = lcpp_llm(
        prompt=prompt,
        temperature= 0.0,
        logprobs=1,
    )
    answer = str(response["choices"][0]["text"]).strip().lower()
    if answer.startswith("yes"):
        output_label = "Not Hallucination"
        prob = 1-float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if answer.startswith("no"):
        output_label = "Hallucination"
        prob = float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if not answer.startswith("no") and not answer.startswith("yes"):
        idx_random = random.randint(0,len(labels)-1)
        output_label = labels[idx_random]
        prob = float(0.5)

    item_to_json = {"label":output_label, "p(Hallucination)":prob}
    if run_on_test:
        item_to_json['id'] = id
    output_json.append(item_to_json)


f = open(path_val_model_agnostic_output, 'w', encoding='utf-8')
json.dump(output_json, f)
f.close()
print("done")

DatasetDict({
    val: Dataset({
        features: ['labels', 'src', 'model', 'hyp', 'task', 'ref', 'tgt', 'label', 'p(Hallucination)'],
        num_rows: 499
    })
})
499


  0%|          | 0/499 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =     205.56 ms
llama_print_timings:      sample time =       8.42 ms /    36 runs   (    0.23 ms per token,  4276.04 tokens per second)
llama_print_timings: prompt eval time =     254.38 ms /    50 tokens (    5.09 ms per token,   196.56 tokens per second)
llama_print_timings:        eval time =    1643.54 ms /    35 runs   (   46.96 ms per token,    21.30 tokens per second)
llama_print_timings:       total time =    2057.28 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =     205.56 ms
llama_print_timings:      sample time =       5.09 ms /    22 runs   (    0.23 ms per token,  4320.50 tokens per second)
llama_print_timings: prompt eval time =     206.36 ms /    36 tokens (    5.73 ms per token,   174.45 tokens per second)
llama_print_timings:        eval time =    1039.12 ms /    21 runs   (   49.48 ms per token,    20.21 tokens per second)
llama_print_timings:       total time =    1

done
